In [9]:
#!/usr/bin/env python
from googleads import adwords

In [22]:
import time

In [33]:
from StringIO import StringIO
import pandas as pd

In [154]:
class get_ads(object):
    def __init__(self, country, start, end):
        self._country = country
        self._client = adwords.AdWordsClient.LoadFromStorage()
        self._page_size = 100
        self._start = start
        self._end = end
    
    def get_accounts(self):
        managed_customer_service = self._client.GetService('ManagedCustomerService', version='v201609')
        selector = {
            'fields': ['CustomerId', 'Name']
        }    
        account_ids = []
        page = managed_customer_service.get(selector)
        if 'entries' in page and page['entries']:
            for account in page['entries']:
                if str(account['name']).startswith(self._country + '_') and 'jobseeker' in str(account['name']) \
                    and '_aed_' not in str(account['name']) and '_app' not in str(account['name'])\
                    and '_display_' not in str(account['name']) and '_content_' not in str(account['name']):              
                    account_ids.append(str(account['customerId']))         
        return account_ids
    
    def get_campaigns(self, account_id):
        self._client.SetClientCustomerId(account_id)
        campaign_service = self._client.GetService('CampaignService', version='v201609')
    
        enabled_campaigns = []
        offset = 0
        qry = 'SELECT Id, Name, Status ORDER BY Name LIMIT {lower}, {upper}'
    
        more_pages = True
        while more_pages:
            page = campaign_service.query(qry.format(lower=offset, upper=self._page_size))
    
            if 'entries' in page:
                for campaign in page['entries']:
                    if campaign['status']=='ENABLED' and campaign['name'].lower().find("dynamic search ads")==-1:
                        enabled_campaigns.append(str(campaign['id']))
    
            offset += self._page_size
            more_pages = offset < int(page['totalNumEntries'])
            time.sleep(1)
        return enabled_campaigns
    
    
    def get_ad_groups(self, account_id, campaign_id):
        self._client.SetClientCustomerId(account_id)
        ad_group_service = self._client.GetService('AdGroupService', version='v201609')
        ad_groups = []
    
        offset = 0
        query = "SELECT Id, BaseAdGroupId, Name, Status WHERE CampaignId= '{0}'".format(campaign_id)
        more_pages = True
        while more_pages:
            page = ad_group_service.query(query + ' LIMIT %s, %s' % (offset, self._page_size))
            if 'entries' in page:
                for ad_group in page['entries']:
                    if ad_group['status'] == 'ENABLED':
                        ad_groups.append(str(ad_group['id']))
            offset += self._page_size
            more_pages = offset < int(page['totalNumEntries'])
        return ad_groups


    def get_ad_data(self, account_id, campaign_id):
        self._client.SetClientCustomerId(account_id)
        report_downloader = self._client.GetReportDownloader(version='v201705')
        qry = ('SELECT CampaignId, AdGroupId, Id, Impressions, Clicks, Cost FROM '
               'AD_PERFORMANCE_REPORT WHERE Status IN [ENABLED] AND CampaignId={2} '
               'DURING {0}, {1}').format(self._start, self._end, campaign_id)  
        stream_data = report_downloader.DownloadReportAsStringWithAwql(qry, 'CSV',
                skip_report_header=True, skip_column_header=False,
                skip_report_summary=True, include_zero_impressions=True)
        report_data = StringIO(stream_data)
        report_df = pd.DataFrame.from_csv(report_data, sep=',').reset_index()
        return report_df
    
    def run(self, account_id, campaign_id):
        data = self.get_ad_data(account_id, campaign_id)
        count_ads = data.groupby(['Campaign ID', 'Ad group ID'])['Ad ID'].count().reset_index()
        #adgroup with greater than 3 ads
        filtered = count_ads[count_ads['Ad ID']>3][['Campaign ID', 'Ad group ID']]
        #add info 
        with_info = pd.merge(filtered, data, how='left', on=['Campaign ID', 'Ad group ID'])
        with_info = with_info[['Campaign ID', 'Ad group ID', 'Ad ID', 'Impressions', 'Clicks']]
        #ctr
        with_info['ctr'] = with_info['Clicks']/with_info['Impressions']
        with_info = with_info.fillna(0)
        return with_info


In [155]:
a = get_ads('uy', '20170601', '20170731')

In [57]:
a.get_accounts()

['9930013120', '6447098408']

In [38]:
a.get_campaigns(6447098408)

['874677791']

In [39]:
len(a.get_ad_groups(6447098408, 874677791))

80

In [60]:
data = a.get_ad_data(6447098408, 874677791)

In [156]:
a.run(6447098408, 874677791)

,Campaign ID,Ad group ID,Ad ID,Impressions,Clicks,ctr
0,874677791,45713167844,205016022122,498,27,0.054217
1,874677791,45713167844,205016021996,506,159,0.314229
2,874677791,45713167844,205016021999,542,149,0.274908
3,874677791,45713167844,205016022125,172,1,0.005814
4,874677791,45713169524,205016022362,0,0,0.000000
5,874677791,45713169524,205016022365,0,0,0.000000
6,874677791,45713169524,205016022236,0,0,0.000000
7,874677791,45713169524,205016022239,0,0,0.000000


In [98]:
ranked = greater_than_3.groupby(['Campaign ID', 'Ad group ID'])['ctr'].rank()

In [103]:
x = greater_than_3.sort_values(['Campaign ID', 'Ad group ID', 'ctr'])

In [117]:
greater_than_3.groupby(['Ad group ID']).aggregate(lambda x: list(x))

,Campaign ID,Ad ID_x,Ad ID_y,Impressions,Clicks,Cost,ctr
Ad group ID,,,,,,,
45713167844,"[874677791, 874677791, 874677791, 874677791]","[4, 4, 4, 4]","[205016021996, 205016021999, 205016022125, 205...","[356, 379, 166, 337]","[110, 107, 0, 19]","[2650000, 2920000, 0, 560000]","[0.308988764045, 0.282321899736, 0.0, 0.056379..."
45713169524,"[874677791, 874677791, 874677791, 874677791]","[4, 4, 4, 4]","[205016022362, 205016022365, 205016022236, 205...","[0, 0, 0, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]","[0.0, 0.0, 0.0, 0.0]"


In [122]:
#greater_than_3.groupby(['Ad group ID']).apply(lambda tdf: pd.Series(dict([[vv,tdf[vv].unique().tolist()] for vv in tdf if vv not in ['A']])  )) 
greater_than_3.groupby(['Campaign ID', 'Ad group ID']).apply(lambda tdf: pd.Series(dict([[vv,tdf[vv].unique().tolist()] for vv in tdf if vv not in ['A']])  )) 


Ad ID_x  \
Campaign ID Ad group ID           
874677791   45713167844     [4]   
            45713169524     [4]   

                                                                   Ad ID_y  \
Campaign ID Ad group ID                                                      
874677791   45713167844  [205016021996, 205016021999, 205016022125, 205...   
            45713169524  [205016022362, 205016022365, 205016022236, 205...   

                           Ad group ID  Campaign ID             Clicks  \
Campaign ID Ad group ID                                                  
874677791   45713167844  [45713167844]  [874677791]  [110, 107, 0, 19]   
            45713169524  [45713169524]  [874677791]                [0]   

                                                  Cost           Impressions  \
Campaign ID Ad group ID                                                        
874677791   45713167844  [2650000, 2920000, 0, 560000]  [356, 379, 166, 337]   
            45713169524                            [0]                   [0]   

                                                                       ctr  
Campaign ID Ad group ID                                                     
874677791   45713167844  [0.308988764045, 0.282321899736, 0.0, 0.056379...  
            45713169524                                              [0.0]

In [105]:
x['ctr'].pct_change()

2         NaN
3         inf
1    4.007499
0    0.094456
4   -1.000000
5         NaN
6         NaN
7         NaN
Name: ctr, dtype: float64